In [2]:
################################  Storage_Optimization.ipynb  ####################################
# Author: Sukhendu Sain
# Description: Main file of codebase. Houses main code
# Data: 23-Nov-2024
#################################################################################

In [3]:
# Import Necessary Libraries, Utils, and Config Files
import utils
from config import *
import pandas as pd
import numpy as np
import os, re
import matplotlib.pyplot as plt
# import importlib
# importlib.reload(utils)

### Config - Parameters/Other Variables/File Paths

In [4]:
## Files Path ##
ROOT_FILE_PATH = '\\'.join(os.getcwd().split('\\')[:-1])

AKINS_FOMO_FILE_PATH = os.path.join(ROOT_FILE_PATH, r"Data&Files\AKINS FoMoCo_Piece_Sales_112222_YTD.xlsx")
GPARTS_FILE_PATH = os.path.join(ROOT_FILE_PATH, r"Data&Files\GPARTS Part Measures.xlsx")
WHOLESALE_FILE_PATH = os.path.join(ROOT_FILE_PATH, r"Data&Files\Wholesale JAN_Oct_Parts_Ranking_Counter_Invoices_All_Brands.xlsx")
SERVICE_FILE_PATH = os.path.join(ROOT_FILE_PATH, r"Data&Files\Service JAN_Oct_Parts_Ranking_ROs_All_Brands.xlsx")
COUNTERPAD_FILE_PATH = os.path.join(ROOT_FILE_PATH, r"Data&Files\Counter_Pad_11142024.xlsx")



## Variables
print_df_after_import = True
print_df_data_analyse = True

# Data Import and Clean

In [ ]:
#### Read FILE:: (AKINS FoMoCo_Piece_Sales_112222_YTD.xlsx) into Dataframe
df_Akins = utils.read_excel(AKINS_FOMO_FILE_PATH)
df_Akins['Part#'] = df_Akins['Part#'].apply(lambda a: "".join(str(a).split('-')))
if print_df_after_import: utils.print_df(df_Akins, 200) # Print the Dataframe
# ~1-2secs

In [ ]:
#### Read FILE:: (GPARTS Part Measures.xlsx) into Dataframe
df_Gparts = utils.read_excel(GPARTS_FILE_PATH)
if print_df_after_import: utils.print_df(df_Gparts) # Print the Dataframe
# ~50-60secs

In [ ]:
#### Read FILE:: (Wholesale JAN_Oct_Parts_Ranking_Counter_Invoices_All_Brands.xlsx) into Dataframe
df_Wholesale = utils.read_excel(WHOLESALE_FILE_PATH)

# Clean the Wholesale Dataframe
df_Wholesale['Description'] = df_Wholesale['Description'].astype(str)
df_Wholesale = df_Wholesale.drop(columns=[col for col in df_Wholesale.columns if 'Unnamed' in col], inplace=False)
df_Wholesale = df_Wholesale[(df_Wholesale['Vendor'] == 'FOR') | (df_Wholesale['Vendor'] == 'CHR')].reset_index()
df_Wholesale.loc[df_Wholesale['Description'].apply(lambda x: len(x.split("      ")) > 1), 'Avg. Cost'] = df_Wholesale['Description'].apply(lambda x: [i for i in x.strip().split("      ")][-1])
df_Wholesale.loc[df_Wholesale['Description'].apply(lambda x: len(x.split("      ")) > 1), 'Description'] = df_Wholesale['Description'].apply(lambda x: "     ".join([i for i in x.strip().split("      ")][:-1]))

if print_df_after_import: utils.print_df(df_Wholesale) # Print the Dataframe
# ~12-15secs

In [ ]:
#### Read FILE:: (Service JAN_Oct_Parts_Ranking_ROs_All_Brands.xlsx) into Dataframe
df_Service = utils.read_excel(SERVICE_FILE_PATH)

# Clean the Service Dataframe
df_Service = df_Service.drop(columns=[col for col in df_Service.columns if 'Unnamed' in col], inplace=False)
df_Service = df_Service[(df_Service['Vendor'] == 'FOR') | (df_Service['Vendor'] == 'CHR')].reset_index()

if print_df_after_import: utils.print_df(df_Service, 100) # Print the Dataframe
# ~5-6secs

In [9]:
#### Read FILE:: (Counter Pad) into Dataframe

# Data Processing & Calculation

In [ ]:
## Make a Big Final Dataframe
# It will have the Columns - 'Part Number', 'Part Type', 'Active', 'Sold (Pcs.)', 'Length/Depth', 'Width', 'Height', 'Zone', 'Storage Type', 'Sub Storage', 'Number of Storage needed'
# It will have all the rows with common part nos. from all 4 Files, having Appropriate Sold Pcs. Values, and Dimensions

main_list = []

gParts_PartNos = set(df_Gparts['Svc Part Number'])

# common_part_numbers = gParts_PartNos & set(df_Akins['Part#'])
# for pn, pt, ac, s, ld, w, h in zip(common_part_numbers, df_Gparts["Svc Part Number Description"], df_Gparts['Is Active?'], df_Akins['Sold Pcs '], df_Gparts['Prod Att - Length'], df_Gparts['Prod Att- Width'], df_Gparts['Prod Att - Height']):
#     main_list.append([pn, pt, ac, s, ld, w, h, "", "", "", ""])

common_part_numbers = gParts_PartNos & set(df_Wholesale['Part Number'])
for pn, pt, ac, s, ld, w, h in zip(common_part_numbers, df_Gparts["Svc Part Number Description"], df_Gparts['Is Active?'], df_Wholesale['Sold'], df_Gparts['Prod Att - Length'], df_Gparts['Prod Att- Width'], df_Gparts['Prod Att - Height']):
    main_list.append([pn, pt, ac, s, ld, w, h, "", "", "", ""])


common_part_numbers = gParts_PartNos & set(df_Service['* indicates a superseded part\nPart Number'])
for pn, pt, ac, s, ld, w, h in zip(common_part_numbers, df_Gparts["Svc Part Number Description"], df_Gparts['Is Active?'], df_Service['Qty Sold'], df_Gparts['Prod Att - Length'], df_Gparts['Prod Att- Width'], df_Gparts['Prod Att - Height']):
    main_list.append([pn, pt, ac, s, ld, w, h, "", "", "", ""])

df_Main = pd.DataFrame(main_list)
df_Main.columns = ['Part#', 'Part Type', 'Active', 'Sold', 'Depth', 'Width', 'Height', 'Zone', 'StorageType', 'SubStorage', 'Num. Storage Required']
df_Main = df_Main.sort_values('Sold', ascending=False)
utils.print_df(df_Main)

In [ ]:
data = []
ssum = 0
totalSoldPCs = int(df_Wholesale["Sold"].sum())
print(ssum, totalSoldPCs)
for i in range(df_Wholesale.shape[0]):
    zone = ""
    if ssum/totalSoldPCs <= 0.2:
        zone = "Red Hot Zone"
    if ssum/totalSoldPCs > 0.2 and ssum/totalSoldPCs <= 0.4:
        zone = "Orange Zone"
    if ssum/totalSoldPCs > 0.4 and ssum/totalSoldPCs <= 0.6:
        zone = "Yellow Zone"
    if ssum/totalSoldPCs > 0.6 and ssum/totalSoldPCs <= 0.8:
        zone = "Green Zone"
    if ssum/totalSoldPCs > 0.8:
        zone = "Blue Zone"
    data.append([df_Wholesale["Part Number"].iloc[i], zone, df_Wholesale["Sold"].iloc[i]])
    ssum = ssum + df_Wholesale["Sold"].iloc[i]
df_zones = pd.DataFrame(data)
utils.print_df(df_zones,None)

with open("htt.txt", "w") as f:
    f.write(df_zones.to_string())

In [ ]:
data = []
sum_list = []
ssum = 0
thres = 0.2 * df_Main["Sold"].sum()
totalSoldPCs = int(df_Main["Sold"].sum())
print(thres, totalSoldPCs)
zones = ['RH', 'OR', 'YE', 'GR', 'BL']
zi = 0
for i in range(df_Main.shape[0]):
    if ssum > thres:     
        for i1 in  range(i):
            data.append([df_Main['Part#'].iloc[i1], zones[zi], df_Main["Sold"].iloc[i1]])
        zi = zi + 1
        ssum = 0
    else:
        ssum = ssum + df_Main["Sold"].iloc[i]
        sum_list.append(ssum)
        
df_zones = pd.DataFrame(data)
print(df_zones.shape[0])
utils.print_df(df_zones)

with open("htt.txt", "w") as f:
    f.write(df_zones.to_string())

In [ ]:
df_Main

In [ ]:
data = []
sum_list = []
ssum = 0
thres = 0.2 * df_Wholesale["Sold"].sum()
totalSoldPCs = int(df_Wholesale["Sold"].sum())
print(thres, totalSoldPCs)
zones = ['RH', 'OR', 'YE', 'GR', 'BL']
zi = 0
for i in range(df_Wholesale.shape[0]):
    if ssum > thres:     
        for i1 in  range(i):
            data.append([df_Wholesale['Part Number'].iloc[i1], zones[zi], df_Wholesale["Sold"].iloc[i1]])
        zi = zi + 1
        ssum = 0
    else:
        ssum = ssum + df_Wholesale["Sold"].iloc[i]
        sum_list.append(ssum)
        
df_zones = pd.DataFrame(data)
utils.print_df(df_zones)

with open("htt.txt", "w") as f:
    f.write(df_zones.to_string())
sum_list[1001:]

In [ ]:
df_Main.groupby('Zone').count()

In [ ]:
df_zones.groupby(1)[2].sum(), df_zones.groupby(1).count()

In [ ]:
## Visualize above No Dimensions Data

# Create the bar graph
plt.figure(figsize=(10, 6)) 
bars = ['Red Hot', 'Orange', 'Yellow']
heights = [7, 105, 704]

plt.bar(bars, heights, width=0.4)
plt.title(f'Num of Parts per Zone (Wholesale+GParts)', fontsize=20)
plt.ylabel('Num of Parts')
plt.xticks(rotation=45, fontsize=16)

# Add labels to each bar
for i, v in enumerate(heights):
    plt.text(i, v, str(round(v, 2)), ha='center', va='bottom')

# Show the legend and display the chart
plt.tight_layout()
plt.show()

In [ ]:
## Visualize above No Dimensions Data

# Create the bar graph
plt.figure(figsize=(10, 6)) 
bars = ['Red Hot', 'Orange', 'Yellow']
heights = [57, 398, 1786]

plt.bar(bars, heights, width=0.4)
plt.title(f'Num of Parts per Zone (Wholesale)', fontsize=20)
plt.ylabel('Num of Parts')
plt.xticks(rotation=45, fontsize=16)

# Add labels to each bar
for i, v in enumerate(heights):
    plt.text(i, v, str(round(v, 2)), ha='center', va='bottom')

# Show the legend and display the chart
plt.tight_layout()
plt.show()

In [ ]:
df_Main.iloc[:7]['Sold'].sum()

In [97]:
df_Main[df_Main['Part#'].isin(df_zones[df_zones[1] == 'RH'][0])].iloc[:, :7]
df_Main[df_Main['Part#'].isin(df_zones[df_zones[1] == 'RH'][0])].iloc[:, :7].to_excel('DetailsofWholesaleRHParta.xlsx', index=False)

In [99]:
df_Main.iloc[:7, :].to_excel('DetailsofWholesaleRHParta.xlsx', index=False)